# Bezug Geodaten von Orten, nach denen Straßen benannt sind

Dieses Skript bezieht die Geodaten der Orte, nach denen Straßen in Dresden benannt sind.

In [1]:
# Bibliotheken laden
import pandas as pd
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
# CSV-Datei einlesen
file_path_persons = '../../data/processed/Data_Masterarbeit - list_named_after_humans.csv'
file_path_all_streets = '../../data/processed/Data_Masterarbeit - list_complete.csv'
file_path_street_layout = "../../data/raw/lhd_strassenzuege.csv"

df_persons = pd.read_csv(file_path_persons, sep=",")
df_all = pd.read_csv(file_path_all_streets, sep=",")
df_streets = pd.read_csv(file_path_street_layout, sep=";")

In [ ]:
# Filter nur Ortsnamen
orte_df = df_all[df_all["Benennungstyp"] == "Ort"].dropna(subset=["benannt nach"])

# Geokodierer einrichten
geolocator = Nominatim(user_agent="dresden-ortsnamen")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Funktion zum Geokodieren
def geocode_ort(name):
    try:
        location = geocode(name)  # kein ", Germany" mehr
        if location:
            return (location.latitude, location.longitude, location.address)
    except:
        return (None, None, None)
    return (None, None, None)

tqdm.pandas()

# Alle eindeutigen Orte geokodieren (Caching)
unique_orte = orte_df["benannt nach"].unique()
geo_cache = {}

for ort in tqdm(unique_orte, desc="Geokodiere Orte"):
    geo_cache[ort] = geocode_ort(ort)

# Mapping anwenden: Straße + Ort + Geodaten
geo_df = orte_df[["Straßenname", "benannt nach"]].rename(columns={"benannt nach": "Ortsname"})
geo_df[["lat", "lon", "adresse"]] = geo_df["Ortsname"].apply(lambda x: pd.Series(geo_cache.get(x)))

# Ergebnis speichern
geo_df.to_csv("../../data/processed/Geodaten_Orte_Straßennamen.csv", index=False)


Geokodiere Orte:   3%|▎         | 18/561 [00:16<09:11,  1.01s/it]

In [6]:
fehlgeschlagen = geo_df[geo_df['lat'].isna()]
print("Nicht gefundene Orte:")
print(fehlgeschlagen['Ortsname'])

Nicht gefundene Orte:
716                Cunnersdorf (Wüstung)
1144    Nasssau (Frauenstein Erzgebirge)
1158                Knappsdorf (Rähnitz)
1240            Ramwoltitz (Johannstadt)
1265              Oberlösnitz (Radebeul)
2170                 Gorknitz (Kreischa)
2204                Moritzburg, Pillnitz
2384              Kleinborthen (Borthen)
Name: Ortsname, dtype: object
